In [25]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

model_name = os.getenv("LLM_MODEL") or "gpt-4o-mini"
model_provider = os.getenv("LLM_MODEL_PROVIDER") or "openai"


In [5]:
loader = PyPDFLoader("../data/unsu.pdf")
pages = loader.load_and_split()

print(pages[0])

page_content='운수 좋은날
현진건
새침하게 흐린 품이 눈이 올 듯하더니 눈은 아니 오고 얼다가 만 비가 추
적추적 내리는 날이었다.
이날이야말로 동소문 안에서 인력거꾼 노릇을 하는 김첨지에게는 오래간만
에도 닥친 운수 좋은 날이었다 문안에 거기도 문밖은 아니지만 들어간답. ( )
시는 앞집 마마님을 전찻길까지 모셔다 드린 것을 비롯으로 행여나 손님이
있을까 하고 정류장에서 어정어정하며 내리는 사람 하나하나에게 거의 비는
듯한 눈결을 보내고 있다가 마침내 교원인 듯한 양복쟁이를 동광학교(東光
까지 태워다 주기로 되었다) .學校
첫 번에 삼십전 둘째 번에 오십전 아침 댓바람에 그리 흉치 않은 일이, -
었다 그야말로 재수가 옴붙어서 근 열흘 동안 돈 구경도 못한 김첨지는 십.
전짜리 백동화 서 푼 또는 다섯 푼이 찰깍 하고 손바닥에 떨어질 제 거의,
눈물을 흘릴 만큼 기뻤었다 더구나 이날 이때에 이 팔십 전이라는 돈이 그.
에게 얼마나 유용한지 몰랐다 컬컬한 목에 모주 한 잔도 적실 수 있거니와.
그보다도 앓는 아내에게 설렁탕 한 그릇도 사다 줄 수 있음이다.
그의 아내가 기침으로 쿨룩거리기는 벌써 달포가 넘었다 조밥도 굶기를.
먹다시피 하는 형편이니 물론 약 한 첩 써본 일이 없다 구태여 쓰려면 못.
쓸 바도 아니로되 그는 병이란 놈에게 약을 주어 보내면 재미를 붙여서 자
꾸 온다는 자기의 신조 에 어디까지 충실하였다 따라서 의사에게 보( ) .信條
인 적이 없으니 무슨 병인지는 알 수 없으되 반듯이 누워 가지고 일어나기
는 새로 모로도 못 눕는 걸 보면 중증은 중증인 듯 병이 이대도록 심해지.
기는 열흘전에 조밥을 먹고 체한 때문이다 그때도 김첨지가 오래간만에 돈.
을 얻어서 좁쌀 한 되와 십 전짜리 나무 한 단을 사다 주었더니 김첨지의
말에 의지하면 그 오라질 년이 천방지축으로 냄비에 대고 끓였다 마음은.
급하고 불길은 달지 않아 채 익지도 않은 것을 그 오라질년이 숟가락은 고
만두고 손으로 움켜서 두 뺨에 주먹덩이 같은 혹이 불거지도록 누가 빼앗

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(pages)

print(texts[0])

page_content='운수 좋은날
현진건
새침하게 흐린 품이 눈이 올 듯하더니 눈은 아니 오고 얼다가 만 비가 추
적추적 내리는 날이었다.
이날이야말로 동소문 안에서 인력거꾼 노릇을 하는 김첨지에게는 오래간만
에도 닥친 운수 좋은 날이었다 문안에 거기도 문밖은 아니지만 들어간답. ( )
시는 앞집 마마님을 전찻길까지 모셔다 드린 것을 비롯으로 행여나 손님이
있을까 하고 정류장에서 어정어정하며 내리는 사람 하나하나에게 거의 비는
듯한 눈결을 보내고 있다가 마침내 교원인 듯한 양복쟁이를 동광학교(東光
까지 태워다 주기로 되었다) .學校' metadata={'producer': 'ezPDF Builder 2006', 'creator': 'ezPDF Builder 2006', 'creationdate': '2010-03-19T13:04:08+09:00', 'moddate': '2010-03-19T13:04:08+09:00', 'source': '../data/unsu.pdf', 'total_pages': 10, 'page': 0, 'page_label': '1'}


In [13]:
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",
)


In [16]:
db = Chroma.from_documents(texts, embeddings_model)

In [18]:
llm = ChatOpenAI(model=model_name, temperature=0)

In [21]:
question = "아내가 먹고 싶어하는 음식은 무엇이야?"

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(),
    llm=llm
)

In [22]:
docs = retriever_from_llm.invoke(question)
print(len(docs))
print(docs)

7
[Document(id='32effaa0-026a-4dd3-b596-e01a00b4d271', metadata={'total_pages': 10, 'page': 1, 'producer': 'ezPDF Builder 2006', 'source': '../data/unsu.pdf', 'moddate': '2010-03-19T13:04:08+09:00', 'page_label': '2', 'creator': 'ezPDF Builder 2006', 'creationdate': '2010-03-19T13:04:08+09:00'}, page_content='이 맺히었다 김첨지의 눈시울도 뜨끈뜨끈하였다. .\n이 환자가 그러고도 먹는 데는 물리지 않았다 사흘 전부터 설렁탕 국물이.\n마시고 싶다고 남편을 졸랐다.\n이런 오라질 년 조밥도 못 먹는 년이 설렁탕은 또 처먹고 지랄병을 하“ ! .\n게.”\n라고 야단을 쳐보았건만 못 사주는 마음이 시원치는 않았다, , .\n인제 설렁탕을 사줄 수도 있다 앓는 어미 곁에서 배고파 보채는 개똥이.\n세살먹이 에게 죽을 사줄 수도 있다 팔십 전을 손에 쥔 김 첨지의 마음( ) -\n은 푼푼하였다.'), Document(id='f4f2f03e-54e4-4743-8bda-458a7809a72a', metadata={'creationdate': '2010-03-19T13:04:08+09:00', 'moddate': '2010-03-19T13:04:08+09:00', 'page_label': '8', 'total_pages': 10, 'creator': 'ezPDF Builder 2006', 'source': '../data/unsu.pdf', 'producer': 'ezPDF Builder 2006', 'page': 7}, page_content='씩 더 먹고 나왔다 궂은비는 의연히 추적추적 내린다. .'), Document(id='bdc57999-19b2-41bd-830e-cbc4dc96fb7c', metadata={'page': 

In [28]:
prompt = hub.pull("rlm/rag-prompt")

In [29]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {"context": retriever_from_llm | format_docs, "question": RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

rag_chain.invoke({"input": question})

'아내가 먹고 싶어하는 음식은 설렁탕입니다. 맥락에서 아내는 설렁탕 국물을 마시고 싶어한다고 언급되어 있습니다. 김첨지는 아내의 병과 배고픔을 걱정하며 설렁탕을 사줄 마음을 품고 있습니다.'